In [1]:
import numpy as np
import pandas as pd

FIP계산에 쓰일 계수인 C는 STATIZ사이트에서 cFIP 크롤링해서 사용

In [2]:
import requests
from bs4 import BeautifulSoup

req = requests.get('http://www.statiz.co.kr/constant.php')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
tables = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div.col-md-12.col-xs-12.col-sm-12.col-lg-9 > div > div > div > div.box-body.no-padding.table-responsive > table')
table = tables[0]

In [3]:
table_html=str(table)
table_df_list=pd.read_html(table_html)
table_df = table_df_list[0]

In [4]:
idx=table_df[0:5]
idx

,연도,wOBA,Scale,wBB,wHBP,w1B,w2B,w3B,wHR,runSB,runCS,R/PA,R/W,cFIP
0,2020,0.345,1.145,0.730,0.759,0.936,1.279,1.588,1.947,0.2,0.467,0.134,10.837,3.630
1,2019,0.335,1.242,0.721,0.752,0.945,1.317,1.653,2.074,0.2,0.417,0.118,9.821,3.403
2,2018,0.351,1.075,0.726,0.753,0.920,1.242,1.533,1.857,0.2,0.492,0.141,11.327,3.833
3,2017,0.351,1.107,0.728,0.756,0.927,1.259,1.558,1.901,0.2,0.476,0.135,11.001,3.735
4,2016,0.362,1.097,0.747,0.774,0.944,1.273,1.569,1.898,0.2,0.495,0.140,11.410,3.792


In [5]:
idx=idx[['연도','cFIP']]
idx

,연도,cFIP
0,2020,3.630
1,2019,3.403
2,2018,3.833
3,2017,3.735
4,2016,3.792


In [6]:
idx.rename(columns = {'연도':'GYEAR'}, inplace = True)
idx

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,GYEAR,cFIP
0,2020,3.630
1,2019,3.403
2,2018,3.833
3,2017,3.735
4,2016,3.792


In [14]:
def add_variables(data, year):
    data = pd.read_csv(data, encoding='CP949')
    
    data['CUM_ER'] = None
    data['CUM_INN2'] = None
    data['CUM_KK'] = None
    data['CUM_BB'] = None
    data['CUM_HIT'] = None
    data['CUM_HR'] = None
    data['CUM_AB'] = None
    data['CUM_SF'] = None
    data['CUM_HP'] = None
    data['CUM_R'] = None
    data['CUM_H2']= None
    data['CUM_H3']= None
    data['GYEAR']=None
    
    data.loc[data['G_ID'].str.contains('2016'),'GYEAR']=2016
    data.loc[data['G_ID'].str.contains('2017'),'GYEAR']=2017
    data.loc[data['G_ID'].str.contains('2018'),'GYEAR']=2018
    data.loc[data['G_ID'].str.contains('2019'),'GYEAR']=2019
    data.loc[data['G_ID'].str.contains('2020'),'GYEAR']=2020
    data=pd.merge(data,idx,on='GYEAR')
    
    data = data.sort_values(by=['P_ID', 'GDAY_DS'])
    data= data.drop('GYEAR', axis=1 )
    
    temp = []
    
    for i in range(250):
        temp.append(list(data['P_ID']).index(data['P_ID'].unique()[i]))
    
    temp.append(len(data))
    
    for i in range(250):
        for j in range(temp[i], temp[i+1]):
            data.iloc[j, 38] = np.cumsum(data.iloc[j, 34])
            data.iloc[j, 39] = np.cumsum(data.iloc[j, 13])
            data.iloc[j, 40] = np.cumsum(data.iloc[j, 28])
            data.iloc[j, 41] = np.cumsum(data.iloc[j, 25])
            data.iloc[j, 42] = np.cumsum(data.iloc[j, 17])
            data.iloc[j, 43] = np.cumsum(data.iloc[j, 20])
            data.iloc[j, 44] = np.cumsum(data.iloc[j, 16])
            data.iloc[j, 45] = np.cumsum(data.iloc[j, 24])
            data.iloc[j, 46] = np.cumsum(data.iloc[j, 27])
            data.iloc[j, 47] = np.cumsum(data.iloc[j, 33])
            data.iloc[j, 48] = np.cumsum(data.iloc[j, 18])
            data.iloc[j, 49] = np.cumsum(data.iloc[j, 19])
    
    data = data.sort_index()
    
    data['ERA'] = 9*data['CUM_ER']/(data['CUM_INN2']/3)
    data['K/9'] = 9*data['CUM_KK']/(data['CUM_INN2']/3)
    data['BB/9'] = 9*data['CUM_BB']/(data['CUM_INN2']/3)
    data['K/BB'] = data['CUM_KK']/data['CUM_BB']
    data['WHIP'] = (data['CUM_HIT']+data['CUM_BB'])/(data['CUM_INN2']/3)
    data['BABIP'] = (data['CUM_HIT']-data['CUM_HR'])/(data['CUM_AB']-data['CUM_KK']-data['CUM_HR']+data['CUM_SF'])
    data['DICE']= 3.00 + (13* data['CUM_HR'] + 3*(data['CUM_BB']+ data['HP'])-2* data['CUM_KK'])/(data['CUM_INN2'] /3 )
    data['FIP'] = data['cFIP'] + (13* data['CUM_HR'] + 3*(data['CUM_BB']+data['CUM_HP']) - 2* data['CUM_KK'])/ (data['CUM_INN2']/3)
    data['kFIP'] = data['cFIP']+ (14* data['CUM_HR'] + 3*(data['CUM_BB']+data['CUM_HP']) -data['CUM_KK'])/ (data['CUM_INN2']/3)
    data['RA9'] = 9 * data['CUM_R'] / (data['CUM_INN2'] / 3)
    data['HR/9'] = 9 * data['CUM_HR'] / (data['CUM_INN2'] / 3)
    data['H/9'] = 9 * data['CUM_HIT'] / (data['CUM_INN2'] / 3)
    data['OAVG'] = data['CUM_HIT']/ data['CUM_AB']
    data['OOBP'] = (data['CUM_HIT'] + data['CUM_BB'] + data['CUM_HP'] ) / (data['CUM_AB'] + data['CUM_BB'] +data['CUM_HP'] + data['CUM_SF'])
    data['OSLG'] = ( data['CUM_HIT'] + 2*data['CUM_H2'] + 3*data['CUM_H3']+ 4*data['CUM_HR'])/data['CUM_AB']
    data['OOPS'] = data['OOBP']+ data['OSLG']

    
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data.to_csv('data_%d.csv' % year)

In [15]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2016.csv', 2016)

In [16]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2017.csv', 2017)

IndexError: index 239 is out of bounds for axis 0 with size 239

In [17]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2018.csv', 2018)

In [18]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2019.csv', 2019)

In [19]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2020.csv', 2020)

IndexError: index 234 is out of bounds for axis 0 with size 234